# Reading the APOE dataframe to this notebook

In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime
import os
import pandas as pd
import gzip
import subprocess

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
df_21 = pd.read_csv(f'APOE_file_44908821.csv')
df_21 = df_21[df_21['a_index'] == 1]
df_21 = df_21[["s","GT"]]
df_21 = df_21.rename(columns = {"s": "person_id" , "GT" : "GT_rs7412"})

df_84 = pd.read_csv(f'APOE_file_44908684.csv')
df_84 = df_84[df_84['a_index'] == 1]
df_84 = df_84[["s","GT"]]
df_84 = df_84.rename(columns = {"s": "person_id" , "GT" : "GT_rs429358"})

In [ ]:
df_APOE = pd.merge(df_21,df_84,on="person_id",how="inner")

In [ ]:
df_APOE.loc[df_APOE["GT_rs7412"] == "0/0", "GT_rs7412"] = 'CC'
df_APOE.loc[df_APOE["GT_rs7412"] == "0/1", "GT_rs7412"] = 'TC'
df_APOE.loc[df_APOE["GT_rs7412"] == "1/1", "GT_rs7412"] = 'TT'

df_APOE.loc[df_APOE["GT_rs429358"] == "0/0", "GT_rs429358"] = 'TT'
df_APOE.loc[df_APOE["GT_rs429358"] == "0/1", "GT_rs429358"] = 'CT'
df_APOE.loc[df_APOE["GT_rs429358"] == "1/1", "GT_rs429358"] = 'CC'

In [ ]:
def APOE_category(row):
    if row["GT_rs7412"] == "CC" and row["GT_rs429358"] == "CC":
        return "e4e4"
    elif row["GT_rs7412"] == "CC" and row["GT_rs429358"] == "CT":
        return "e3e4"
    elif row["GT_rs7412"] == "TC" and row["GT_rs429358"] == "CT":
        return "e2e4"
    elif row["GT_rs7412"] == "CC" and row["GT_rs429358"] == "TT":
        return "e3e3"
    elif row["GT_rs7412"] == "TC" and row["GT_rs429358"] == "TT":
        return "e2e3"
    elif row["GT_rs7412"] == "TT" and row["GT_rs429358"] == "TT":
        return "e2e2"
    else:
        return "d"

df_APOE["APOE_group"] = df_APOE.apply(APOE_category, axis=1)

df_APOE = df_APOE[["person_id","APOE_group"]]

df_APOE = df_APOE[df_APOE["APOE_group"] != "d"]

# Patients and Demographics

In [ ]:
import pandas
import os

# This query represents dataset "APOE_April2024" for domain "person" and was generated for All of Us Controlled Tier Dataset v6
dataset_49198409_person_sql = """
    SELECT
        person.person_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                    UNION
                    DISTINCT SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_array_data = 1 
                ) 
        )"""

dataset_49198409_person_df = pandas.read_gbq(
    dataset_49198409_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook") 

dataset_49198409_person_df = dataset_49198409_person_df.drop_duplicates(subset=['person_id'])
dataset_49198409_person_df['date_of_birth'] = dataset_49198409_person_df['date_of_birth'].dt.date

dataset_49198409_person_df.head(5)

In [ ]:
def Gender_modify(row):
    if row['gender'] == "Male" or row["sex_at_birth"] == "Male":
        return "Male"
    else:
        return "Female"

# Apply the function to the DataFrame
dataset_49198409_person_df['Gender_modified'] = dataset_49198409_person_df.apply(Gender_modify, axis=1)
dataset_49198409_person_df = dataset_49198409_person_df.drop(columns=['gender']).drop(columns=['sex_at_birth'])

In [ ]:
def Race_modify(row):
    if row['race'] == "White":
        return "White"
    elif row['race'] == "Black or African American":
        return "Black or African American"
    else:
        return "White"

# Apply the function to the DataFrame
dataset_49198409_person_df['Race_modified'] = dataset_49198409_person_df.apply(Race_modify, axis=1)
dataset_49198409_person_df = dataset_49198409_person_df.drop(columns=['race']).drop(columns=['ethnicity'])

In [ ]:
df_APOE_person = pd.merge(df_APOE,dataset_49198409_person_df,on="person_id",how="inner")
df_APOE_person.head()

In [ ]:
my_dataframe = df_APOE_person

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_Person.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Drug dataframes

In [ ]:
import pandas
import os

# This query represents dataset "APOE_April2024" for domain "drug" and was generated for All of Us Controlled Tier Dataset v6
dataset_49198409_drug_sql = """
    SELECT
        d_exposure.person_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_source_concept.concept_name as source_concept_name
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (
                    SELECT
                        DISTINCT ca.descendant_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                    JOIN
                        (
                            SELECT
                                DISTINCT c.concept_id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c       
                            JOIN
                                (
                                    SELECT
                                        CAST(cr.id as string) AS id             
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr             
                                    WHERE
                                        concept_id IN (
                                            21600438, 21600451, 21601664, 21601744, 21601782, 21601853
                                        )             
                                        AND full_text LIKE '%_rank1]%'       
                                ) a 
                                    ON (
                                        c.path LIKE CONCAT('%.',
                                    a.id,
                                    '.%') 
                                    OR c.path LIKE CONCAT('%.',
                                    a.id) 
                                    OR c.path LIKE CONCAT(a.id,
                                    '.%') 
                                    OR c.path = a.id) 
                                WHERE
                                    is_standard = 1 
                                    AND is_selectable = 1
                                ) b 
                                    ON (
                                        ca.ancestor_id = b.concept_id
                                    )
                            )
                        )  
                        AND (
                            d_exposure.PERSON_ID IN (
                                SELECT
                                    distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_whole_genome_variant = 1 
                                    UNION
                                    DISTINCT SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_array_data = 1 
                                ) 
                        )
                    )
            ) d_exposure 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id"""

dataset_49198409_drug_df = pandas.read_gbq(
    dataset_49198409_drug_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

df_APOE_person_drug = pd.merge(df_APOE,dataset_49198409_drug_df,on="person_id",how="inner")
df_APOE_person_drug.head(5)

In [ ]:
import re

def statin_use(row):
    # Dictionaries to map statin drugs to their dosage and associated risk level
    statin_mapping = {
        'atorvastatin': {'10': 'Moderate', '20': 'Moderate', '40': 'High', '80': 'High', 'base': 'Moderate'},
        'fluvastatin': {'20': 'Low', '40': 'Low', '80': 'Moderate', 'base': 'Low'},
        'lovastatin': {'10': 'Low', '20': 'Low', '40': 'Moderate', '60': 'Moderate', '80': 'Moderate', 'base': 'Low'},
        'pitavastatin': {'base': 'Moderate'},
        'pravastatin': {'sodium 10': 'Low', 'sodium 20': 'Low', 'sodium 40': 'Moderate', 'sodium 80': 'Moderate', 'sodium 30': 'Moderate', 'base': 'Low'},
        'rosuvastatin': {'5': 'Moderate', '10': 'Moderate', '20': 'High', '40': 'High', 'calcium 5': 'Moderate', 'calcium 10': 'Moderate', 'calcium 20': 'High', 'calcium 40': 'High', 'base': 'Moderate'},
        'simvastatin': {'5': 'Low', '10': 'Low', '20': 'Moderate', '40': 'Moderate', '80': 'High', 'base': 'Low'}
    }
    
    # Iterate through each drug and determine statin use
    for drug, doses in statin_mapping.items():
        if drug in row['standard_concept_name']:
            for dose, level in doses.items():
                if dose == 'base':  # Handle the base case (no specific dosage mentioned)
                    if not re.search(r'\d', row['standard_concept_name']):
                        return level
                elif f"{drug} {dose}" in row['standard_concept_name']:
                    return level
    return "No"  # Default case if no conditions are met

# Apply the function to the DataFrame
df_APOE_person_drug['statin_use'] = df_APOE_person_drug.apply(statin_use, axis=1)

In [ ]:
import re
import pandas as pd

# List of hypertension medications
Hypertension_meds = [
    "verapamil", "valsartan", "trandolapril", "telmisartan", "riociguat", "reserpine", "ramipril", "quinapril", "propranolol",
    "prazosin", "polythiazide", "pindolol", "perindopril", "penbutolol", "olmesartan", "nisoldipine", "nifedipine", "nicardipine",
    "nebivolol", "nadolol", "moexipril", "metoprolol", "methyldopa", "macitentan", "losartan", "lisinopril", "latanoprost", "labetalol",
    "isradipine", "irbesartan", "indapamide", "hydrochlorothiazide", "hydralazine", "fosinopril", "felodipine", "eprosartan", "enalapril",
    "dorzolamide", "diltiazem", "clonidine", "clevidipine", "chlorthalidone", "chlorothiazide", "celiprolol", "carvedilol", "carteolol",
    "captopril", "candesartan", "bosentan", "bisoprolol", "betaxolol", "bendroflumethiazide", "benazepril", "azilsartan", "atenolol",
    "amlodipine", "amiloride", "ambrisentan", "aliskiren", "acebutolol", "tadalafil"
]

# Regular expression pattern for medications
meds_pattern = '|'.join(Hypertension_meds)

# Update the DataFrame
df_APOE_person_drug['Hypertension_Med_Used'] = df_APOE_person_drug['standard_concept_name'].str.contains(meds_pattern).replace({True: 'Yes', False: 'No'})

In [ ]:
df_APOE_person_drug['Aspirin_Used'] = df_APOE_person_drug['standard_concept_name'].str.contains("aspirin").replace({True: 'Yes', False: 'No'})

In [ ]:
#statin_grouping
statin_table = df_APOE_person_drug[df_APOE_person_drug["statin_use"] != "No"]

statin_table = statin_table.groupby(['person_id',"statin_use"]).agg({'drug_exposure_start_datetime': 'min', 'drug_exposure_end_datetime': 'max'}).reset_index()

statin_table = statin_table.rename(columns={'drug_exposure_start_datetime': 'Min_statin_start', 'drug_exposure_end_datetime': 'Max_statin_end'})

statin_table = statin_table.drop_duplicates()

In [ ]:
#Hypertension_grouping
Hypertension_table = df_APOE_person_drug[df_APOE_person_drug["Hypertension_Med_Used"] != "No"]

Hypertension_table = Hypertension_table.groupby(['person_id',"Hypertension_Med_Used"]).agg({'drug_exposure_start_datetime': 'min', 'drug_exposure_end_datetime': 'max'}).reset_index()
Hypertension_table = Hypertension_table.rename(columns={'drug_exposure_start_datetime': 'Min_AntiHyper_start', 'drug_exposure_end_datetime': 'Max_Antihyper_end'})
Hypertension_table = Hypertension_table.drop_duplicates()

In [ ]:
#Aspirin_grouping
Aspirin_table = df_APOE_person_drug[df_APOE_person_drug["Aspirin_Used"] != "No"]

Aspirin_table = Aspirin_table.groupby(['person_id',"Aspirin_Used"]).agg({'drug_exposure_start_datetime': 'min', 'drug_exposure_end_datetime': 'max'}).reset_index()
Aspirin_table = Aspirin_table.rename(columns={'drug_exposure_start_datetime': 'Min_Aspirin_start', 'drug_exposure_end_datetime': 'Max_Aspirin_end'})
Aspirin_table = Aspirin_table.drop_duplicates()

In [ ]:
df_APOE_demo_drug = pd.merge(df_APOE,statin_table,on="person_id",how="left")
df_APOE_demo_drug = pd.merge(df_APOE_demo_drug,Hypertension_table,on="person_id",how="left")
df_APOE_demo_drug = pd.merge(df_APOE_demo_drug,Aspirin_table,on="person_id",how="left")

In [ ]:
df_APOE_demo_drug['Min_statin_start'] = df_APOE_demo_drug['Min_statin_start'].dt.date
df_APOE_demo_drug['Max_statin_end'] = df_APOE_demo_drug['Max_statin_end'].dt.date

df_APOE_demo_drug['Min_AntiHyper_start'] = df_APOE_demo_drug['Min_AntiHyper_start'].dt.date
df_APOE_demo_drug['Max_Antihyper_end'] = df_APOE_demo_drug['Max_Antihyper_end'].dt.date

df_APOE_demo_drug['Min_Aspirin_start'] = df_APOE_demo_drug['Min_Aspirin_start'].dt.date
df_APOE_demo_drug['Max_Aspirin_end'] = df_APOE_demo_drug['Max_Aspirin_end'].dt.date

df_APOE_demo_drug.head()

In [ ]:
my_dataframe = df_APOE_demo_drug

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_demo_drug.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Labs

In [ ]:
import pandas
import os

# This query represents dataset "APOE_April2024" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v6
dataset_49198409_measurement_sql = """
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.value_as_number
        
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (
                                    3004249, 37053746, 40772572, 40772590
                                )       
                                AND full_text LIKE '%_rank1]%'      
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    measurement.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_whole_genome_variant = 1 
                                UNION
                                DISTINCT SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    has_array_data = 1 
                            ) 
                    )
                )
        ) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id"""

dataset_49198409_measurement_df = pandas.read_gbq(
    dataset_49198409_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

df_APOE_person_drug_labs = pd.merge(df_APOE,dataset_49198409_measurement_df,on="person_id",how="inner")
df_APOE_person_drug_labs['measurement_date'] = df_APOE_person_drug_labs['measurement_datetime'].dt.date

df_APOE_person_drug_labs.head(5)

In [ ]:
import pandas as pd

# Define conditions
condition_cholesterol = (
    df_APOE_person_drug_labs['standard_concept_name'].str.contains('Cholesterol \[Mass/volume\]', case=False) & 
    df_APOE_person_drug_labs['standard_concept_name'].str.contains('serum', case=False)
)

condition_hdl = (
    df_APOE_person_drug_labs['standard_concept_name'].str.contains('HDL', case=False) &
    df_APOE_person_drug_labs['standard_concept_name'].str.contains('\[Mass/volume\]', case=False) &
    df_APOE_person_drug_labs['standard_concept_name'].str.contains('serum', case=False) &
    ~df_APOE_person_drug_labs['standard_concept_name'].str.contains('non', case=False) &
    ~df_APOE_person_drug_labs['standard_concept_name'].str.contains('ratio', case=False)
)

# Extract and combine Cholesterol and HDL values
cholesterol_data = df_APOE_person_drug_labs.loc[condition_cholesterol, ['person_id', 'value_as_number', 'measurement_date']]
cholesterol_data.columns = ['person_id', 'Total_Cholesterol', 'Total_Cholesterol_time']

hdl_data = df_APOE_person_drug_labs.loc[condition_hdl, ['person_id', 'value_as_number', 'measurement_date']]
hdl_data.columns = ['person_id', 'HDL', 'HDL_time']

combined_df = pd.merge(cholesterol_data, hdl_data, on='person_id')

# Filter rows where Total_Cholesterol_time equals HDL_time
filtered_combined_df = combined_df[combined_df['Total_Cholesterol_time'] == combined_df['HDL_time']]

In [ ]:
my_dataframe = filtered_combined_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_HDL_TotalChol.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

In [ ]:
# Extract and merge SBP values
condition_SBP = df_APOE_person_drug_labs['standard_concept_name'].str.contains('Systolic blood pressure', case=False)
SBP_data = df_APOE_person_drug_labs.loc[condition_SBP, ["person_id",'value_as_number', 'measurement_date']]
SBP_data.columns = ['person_id','SBP', 'SBP_time']
SBP_data = SBP_data.drop_duplicates()

SBP_data = SBP_data[SBP_data["SBP"].notnull()]
SBP_data = SBP_data[SBP_data["SBP"]>=20]

In [ ]:
# Merge the dataframes
df_HDL_TotalChol = pd.read_csv(f'APOE_file_HDL_TotalChol.csv')
df_HDL_TotalChol['Total_Cholesterol_time'] = pd.to_datetime(df_HDL_TotalChol['Total_Cholesterol_time'])

SBP_filtered_combined_df = pd.merge(df_HDL_TotalChol, SBP_data, on="person_id", how="inner")
SBP_filtered_combined_df['SBP_time'] = pd.to_datetime(SBP_filtered_combined_df['SBP_time'])

# Calculate the time difference
SBP_filtered_combined_df['time_difference'] = abs(SBP_filtered_combined_df['SBP_time'] - SBP_filtered_combined_df['Total_Cholesterol_time'])

# Filter based on the time condition within 30 days
# time_delta = pd.Timedelta(days=30)
# SBP_filtered_combined_df = SBP_filtered_combined_df[
#     (SBP_filtered_combined_df['SBP_time'] <= SBP_filtered_combined_df['Total_Cholesterol_time'] + time_delta) &
#     (SBP_filtered_combined_df['SBP_time'] >= SBP_filtered_combined_df['Total_Cholesterol_time'] - time_delta)
# ]

# Sort by person_id and time_difference
SBP_filtered_combined_df = SBP_filtered_combined_df.sort_values(by=['person_id', 'time_difference'])

# Drop duplicates to keep only the closest SBP_time for each Total_Cholesterol_time
SBP_filtered_combined_df = SBP_filtered_combined_df.drop_duplicates(subset=['person_id', 'Total_Cholesterol_time'], keep='first')

# Drop the time_difference column as it is no longer needed
SBP_filtered_combined_df = SBP_filtered_combined_df.drop(columns=['time_difference'])


In [ ]:
my_dataframe = SBP_filtered_combined_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_labs_SBP.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Smoking

In [ ]:
import pandas
import os

# This query represents dataset "APOE_April2024" for domain "observation" and was generated for All of Us Controlled Tier Dataset v6
dataset_49198409_observation_sql = """
    SELECT
        observation.person_id,
        o_standard_concept.concept_name as standard_concept_name,
        observation.observation_datetime,
        o_value.concept_name as value_as_concept_name
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
        WHERE
            (
                observation_concept_id IN (
                    4144271, 4144272, 4222303, 42528924, 4276526, 4298794, 43054909, 4310250, 762500
                )
            )  
            AND (
                observation.PERSON_ID IN (
                    SELECT
                        distinct person_id  
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                    WHERE
                        cb_search_person.person_id IN (
                            SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_whole_genome_variant = 1 
                            UNION
                            DISTINCT SELECT
                                person_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                            WHERE
                                has_array_data = 1 
                        ) 
                )
            )
    ) observation 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_standard_concept 
        ON observation.observation_concept_id = o_standard_concept.concept_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_type 
        ON observation.observation_type_concept_id = o_type.concept_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_value 
        ON observation.value_as_concept_id = o_value.concept_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_qualifier 
        ON observation.qualifier_concept_id = o_qualifier.concept_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_unit 
        ON observation.unit_concept_id = o_unit.concept_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
        ON observation.visit_occurrence_id = v.visit_occurrence_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_visit 
        ON v.visit_concept_id = o_visit.concept_id 
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + """.concept` o_source_concept 
        ON observation.observation_source_concept_id = o_source_concept.concept_id"""

dataset_49198409_observation_df = pandas.read_gbq(
    dataset_49198409_observation_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

df_APOE_person_smoking = pd.merge(df_APOE,dataset_49198409_observation_df,on="person_id",how="inner")
df_APOE_person_smoking['observation_datetime'] = df_APOE_person_smoking['observation_datetime'].dt.date

df_APOE_person_smoking.head(5)

In [ ]:
# Define smoker and non-smoker categories
smoker_keywords = [
    'Cigarette smoker', 'Current every day smoker', 'Current some day smoker', 
    'Heavy tobacco smoker', 'Light tobacco smoker', 'Occasional tobacco smoker', 'Smoker', 
    'Smoker, current status unknown', 'Smokes tobacco daily', 'Chews tobacco', 'Cigar smoker', 
    'Pipe smoker', 'Snuff tobacco', 'Cigars',"Yes",'Pipe','Medications','Maybe'
]

non_smoker_keywords = [
    'No matching concept', 'Never smoked tobacco', 'Never smoker', 'Non-smoker', 'Unknown if ever smoked', 
    'Unable to assess', 'Don\'t know', 'None', 'No','Former smoker','Ex-smoker','Don\'t know'
]

# Categorize the data
df_APOE_person_smoking['smoker'] = df_APOE_person_smoking['value_as_concept_name'].apply(
    lambda x: 'smoker' if x in smoker_keywords else 'non-smoker'
)

df_APOE_person_smoking = df_APOE_person_smoking.drop(columns=['standard_concept_name','value_as_concept_name'])

In [ ]:
my_dataframe = df_APOE_person_smoking

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_smoking.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Diabetes

In [ ]:
import pandas
import os

# This query represents dataset "APOE_April2024" for domain "condition" and was generated for All of Us Controlled Tier Dataset v6
dataset_87704195_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (201254, 201820, 201826, 4008576, 4193704, 443412)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                    UNION
                    DISTINCT SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_array_data = 1 ) )
                )
            ) c_occurrence 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                ON c_occurrence.condition_type_concept_id = c_type.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                ON v.visit_concept_id = visit.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_87704195_condition_df = pandas.read_gbq(
    dataset_87704195_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

df_APOE_person_Diabetes = pd.merge(df_APOE,dataset_87704195_condition_df,on="person_id",how="inner")
df_APOE_person_Diabetes['condition_start_datetime'] = df_APOE_person_Diabetes['condition_start_datetime'].dt.date

df_APOE_person_Diabetes.head(5)

In [ ]:
condition_Diabetes = (
    ~df_APOE_person_Diabetes['standard_concept_name'].str.lower().str.contains('during pregnancy', case=False) & 
    ~df_APOE_person_Diabetes['standard_concept_name'].str.lower().str.contains('neonatal', case=False) & 
    ~df_APOE_person_Diabetes['standard_concept_name'].str.lower().str.contains('gestational', case=False) &
    ~df_APOE_person_Diabetes['standard_concept_name'].str.lower().str.contains('puerperium', case=False) &
    ~df_APOE_person_Diabetes['standard_concept_name'].str.lower().str.contains('in pregnancy', case=False)
)


Diabetes_data = df_APOE_person_Diabetes.loc[condition_Diabetes, ['person_id', 'standard_concept_name', 'condition_start_datetime']]
Diabetes_data.columns = ['person_id', 'standard_concept_name', 'condition_start_datetime']

In [ ]:
Diabetes_data = Diabetes_data.groupby(['person_id']).agg({'condition_start_datetime': 'min'}).reset_index()
Diabetes_data = Diabetes_data.rename(columns={'condition_start_datetime': 'Diabetes_start_date'})
Diabetes_data['Diabetes'] = "1"
Diabetes_data = Diabetes_data.drop_duplicates()

df_APOE_person_diabetes = pd.merge(df_APOE,Diabetes_data,on="person_id",how="left")
df_APOE_person_diabetes['Diabetes'] = df_APOE_person_diabetes['Diabetes'].fillna("0")

In [ ]:
my_dataframe = df_APOE_person_diabetes

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_Diabetes.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Final edits

In [ ]:
# Read the CSV files into DataFrames
df_demo = pd.read_csv('APOE_file_Person.csv')
df_demo = df_demo.drop_duplicates()
print(df_demo.shape[0])
print(df_demo['person_id'].nunique())

df_labs = pd.read_csv('APOE_file_labs_SBP.csv')

print(df_labs.shape[0])
print(df_labs['person_id'].nunique())

df_demo_labs = df_demo.merge(df_labs, on='person_id', how='inner')
df_demo_labs = df_demo_labs.drop_duplicates()
print(df_demo_labs.shape[0])
print(df_demo_labs['person_id'].nunique())
df_demo_labs.head()

In [ ]:
import pandas as pd
from datetime import timedelta

df_smoking = pd.read_csv('APOE_file_smoking.csv').drop(columns=['APOE_group'])
print(df_smoking.shape[0])
print(df_smoking['person_id'].nunique())

# Perform inner joins
df_demo_lab_smoking = df_demo_labs.merge(df_smoking, on='person_id', how='left')

df_demo_lab_smoking['Total_Cholesterol_time'] = pd.to_datetime(df_demo_lab_smoking['Total_Cholesterol_time'])
df_demo_lab_smoking['observation_datetime'] = pd.to_datetime(df_demo_lab_smoking['observation_datetime'])

# Calculate the time difference
df_demo_lab_smoking['time_difference'] = abs(df_demo_lab_smoking['observation_datetime'] - df_demo_lab_smoking['Total_Cholesterol_time'])

# Sort by person_id and time_difference
df_demo_lab_smoking = df_demo_lab_smoking.sort_values(by=['person_id','Total_Cholesterol_time' ,'time_difference'])

# Drop duplicates to keep only the closest SBP_time for each Total_Cholesterol_time
df_demo_lab_smoking = df_demo_lab_smoking.drop_duplicates(subset=['person_id', 'Total_Cholesterol_time'], keep='first')
print(df_demo_lab_smoking.shape[0])
print(df_demo_lab_smoking['person_id'].nunique())

df_demo_lab_smoking = df_demo_lab_smoking.drop(columns=["observation_datetime",'time_difference'])

df_demo_lab_smoking.head()

In [ ]:
import pandas as pd
from datetime import timedelta
import numpy as np

df_drug = pd.read_csv('APOE_file_demo_drug.csv')
df_drug= df_drug[['person_id', 'statin_use', 'Min_statin_start', 'Max_statin_end']]
df_drug = df_drug.drop_duplicates()
print(df_drug.shape[0])
print(df_drug['person_id'].nunique())

df_drug['Min_statin_start'] = pd.to_datetime(df_drug['Min_statin_start'])
df_drug['Max_statin_end'] = pd.to_datetime(df_drug['Max_statin_end'])

# Perform joins
print(df_demo_lab_smoking.shape[0])
print(df_demo_lab_smoking['person_id'].nunique())

df_demo_lab_smoking_drugs = df_demo_lab_smoking.merge(df_drug, on='person_id', how='left')
print(df_demo_lab_smoking_drugs.shape[0])
print(df_demo_lab_smoking_drugs['person_id'].nunique())

# Check if the statin start date is after the cholesterol date and replace the columns with null values
condition = df_demo_lab_smoking_drugs['Min_statin_start'] >= df_demo_lab_smoking_drugs['Total_Cholesterol_time']
df_demo_lab_smoking_drugs.loc[condition, ['statin_use', 'Min_statin_start', 'Max_statin_end']] = np.nan

# Check if both statin start and end dates are less than the cholesterol date and replace the columns with null values
condition = (df_demo_lab_smoking_drugs['Min_statin_start'] < df_demo_lab_smoking_drugs['Total_Cholesterol_time']) & (df_demo_lab_smoking_drugs['Max_statin_end'] < df_demo_lab_smoking_drugs['Total_Cholesterol_time'])
df_demo_lab_smoking_drugs.loc[condition, ['statin_use', 'Min_statin_start', 'Max_statin_end']] = np.nan

df_demo_lab_smoking_drugs = df_demo_lab_smoking_drugs.drop_duplicates()

df_demo_lab_smoking_drugs = df_demo_lab_smoking_drugs[
    ((df_demo_lab_smoking_drugs['Min_statin_start'] < df_demo_lab_smoking_drugs['Total_Cholesterol_time']) | 
     (df_demo_lab_smoking_drugs['Min_statin_start'].isnull())) &
    ((df_demo_lab_smoking_drugs['Max_statin_end'] >= df_demo_lab_smoking_drugs['Total_Cholesterol_time']) | 
     (df_demo_lab_smoking_drugs['Max_statin_end'].isnull()))
]

print(df_demo_lab_smoking_drugs.shape[0])
print(df_demo_lab_smoking_drugs['person_id'].nunique())

# Define a function to map statin doses to numeric values
def map_dose(dose):
    dose_mapping = {'Low': 1, 'Moderate': 2, 'High': 3}
    return dose_mapping.get(dose, 0)

# Apply the dose mapping
df_demo_lab_smoking_drugs['statin_dose_value'] = df_demo_lab_smoking_drugs['statin_use'].apply(map_dose)

# Sort by person_id, Total_Cholesterol_time, and dose_value
df_demo_lab_smoking_drugs = df_demo_lab_smoking_drugs.sort_values(by=['person_id', 'Total_Cholesterol_time', 'statin_dose_value'], ascending=[True, True, False])

# Drop duplicates to keep only the highest dose statin for each Total_Cholesterol_time
df_demo_lab_smoking_drugs = df_demo_lab_smoking_drugs.drop_duplicates(subset=['person_id', 'Total_Cholesterol_time'], keep='first')

# Drop the unnecessary columns
df_demo_lab_smoking_drugs = df_demo_lab_smoking_drugs.drop(columns=['Min_statin_start','Max_statin_end', 'statin_dose_value'])

print(df_demo_lab_smoking_drugs.shape[0])
print(df_demo_lab_smoking_drugs['person_id'].nunique())

# Display the first few rows of the final DataFrame
df_demo_lab_smoking_drugs.head()


In [ ]:
import pandas as pd
from datetime import timedelta
import numpy as np

df_drug = pd.read_csv('APOE_file_demo_drug.csv')
df_drug= df_drug[['person_id', 'Hypertension_Med_Used','Min_AntiHyper_start','Max_Antihyper_end']]
df_drug = df_drug.drop_duplicates()
print(df_drug.shape[0])
print(df_drug['person_id'].nunique())

df_drug['Min_AntiHyper_start'] = pd.to_datetime(df_drug['Min_AntiHyper_start'])
df_drug['Max_Antihyper_end'] = pd.to_datetime(df_drug['Max_Antihyper_end'])

# Perform joins
print(df_demo_lab_smoking_drugs.shape[0])
print(df_demo_lab_smoking_drugs['person_id'].nunique())

df_demo_lab_smoking_drugs_AntiH = df_demo_lab_smoking_drugs.merge(df_drug, on='person_id', how='left')
print(df_demo_lab_smoking_drugs_AntiH.shape[0])
print(df_demo_lab_smoking_drugs_AntiH['person_id'].nunique())

# Check if the statin start date is after the cholesterol date and replace the columns with null values
condition = df_demo_lab_smoking_drugs_AntiH['Min_AntiHyper_start'] >= df_demo_lab_smoking_drugs_AntiH['Total_Cholesterol_time']
df_demo_lab_smoking_drugs_AntiH.loc[condition, ['Hypertension_Med_Used','Min_AntiHyper_start','Max_Antihyper_end']] = np.nan

# Check if both statin start and end dates are less than the cholesterol date and replace the columns with null values
condition = (df_demo_lab_smoking_drugs_AntiH['Min_AntiHyper_start'] < df_demo_lab_smoking_drugs_AntiH['Total_Cholesterol_time']) & (df_demo_lab_smoking_drugs_AntiH['Max_Antihyper_end'] < df_demo_lab_smoking_drugs_AntiH['Total_Cholesterol_time'])
df_demo_lab_smoking_drugs_AntiH.loc[condition, ['Hypertension_Med_Used','Min_AntiHyper_start','Max_Antihyper_end']] = np.nan

df_demo_lab_smoking_drugs_AntiH = df_demo_lab_smoking_drugs_AntiH.drop_duplicates()

df_demo_lab_smoking_drugs_AntiH = df_demo_lab_smoking_drugs_AntiH[
    ((df_demo_lab_smoking_drugs_AntiH['Min_AntiHyper_start'] < df_demo_lab_smoking_drugs_AntiH['Total_Cholesterol_time']) | 
     (df_demo_lab_smoking_drugs_AntiH['Min_AntiHyper_start'].isnull())) &
    ((df_demo_lab_smoking_drugs_AntiH['Max_Antihyper_end'] >= df_demo_lab_smoking_drugs_AntiH['Total_Cholesterol_time']) | 
     (df_demo_lab_smoking_drugs_AntiH['Max_Antihyper_end'].isnull()))
]

print(df_demo_lab_smoking_drugs_AntiH.shape[0])
print(df_demo_lab_smoking_drugs_AntiH['person_id'].nunique())

df_demo_lab_smoking_drugs_AntiH.head()

In [ ]:
df_Diabetes = pd.read_csv('APOE_file_Diabetes.csv').drop(columns=['APOE_group'])
print(df_Diabetes.shape[0])
print(df_Diabetes['person_id'].nunique())

# Specify the common column for joining, assuming it's 'PersonID'
common_column = 'person_id'

# Perform inner joins
df_demo_lab_smoking_drugs_AntiH_Diab = df_demo_lab_smoking_drugs_AntiH.merge(df_Diabetes, on=common_column, how='left')

print(df_demo_lab_smoking_drugs_AntiH_Diab.shape[0])
print(df_demo_lab_smoking_drugs_AntiH_Diab['person_id'].nunique())

df_demo_lab_smoking_drugs_AntiH_Diab.head()

In [ ]:
import pandas as pd

# Ensure the date columns are in datetime format
df_demo_lab_smoking_drugs_AntiH_Diab['date_of_birth'] = pd.to_datetime(df_demo_lab_smoking_drugs_AntiH_Diab['date_of_birth'])
df_demo_lab_smoking_drugs_AntiH_Diab['Total_Cholesterol_time'] = pd.to_datetime(df_demo_lab_smoking_drugs_AntiH_Diab['Total_Cholesterol_time'])

# Calculate the age in years
df_demo_lab_smoking_drugs_AntiH_Diab['Age'] = (df_demo_lab_smoking_drugs_AntiH_Diab['Total_Cholesterol_time'] - df_demo_lab_smoking_drugs_AntiH_Diab['date_of_birth']).dt.days // 365.25

df_demo_lab_smoking_drugs_AntiH_Diab.head()


In [ ]:
# Ensure the date columns are in datetime format
df_demo_lab_smoking_drugs_AntiH_Diab['Diabetes_start_date'] = pd.to_datetime(df_demo_lab_smoking_drugs_AntiH_Diab['Diabetes_start_date'])

# Apply the condition row-wise
df_demo_lab_smoking_drugs_AntiH_Diab['Diabetes_at_timeLab'] = df_demo_lab_smoking_drugs_AntiH_Diab.apply(
    lambda row: '1' if row['Diabetes_start_date'] <= row['Total_Cholesterol_time'] else '0',
    axis=1
)

print(df_demo_lab_smoking_drugs_AntiH_Diab.shape[0])
print(df_demo_lab_smoking_drugs_AntiH_Diab['person_id'].nunique())
df_demo_lab_smoking_drugs_AntiH_Diab.head()

In [ ]:
df_demo_lab_smoking_drugs_AntiH_Diab['Hypertension_Med_Used'] = df_demo_lab_smoking_drugs_AntiH_Diab['Hypertension_Med_Used'].apply(lambda x: "1" if x == 'Yes' else "0")
df_demo_lab_smoking_drugs_AntiH_Diab['smoker'] = df_demo_lab_smoking_drugs_AntiH_Diab['smoker'].apply(lambda x: "1" if x == 'Yes' else "0")

In [ ]:
df_demo_lab_smoking_drugs_AntiH_Diab = df_demo_lab_smoking_drugs_AntiH_Diab[df_demo_lab_smoking_drugs_AntiH_Diab["Total_Cholesterol"]>1]
df_demo_lab_smoking_drugs_AntiH_Diab = df_demo_lab_smoking_drugs_AntiH_Diab[df_demo_lab_smoking_drugs_AntiH_Diab["HDL"]>1]


In [ ]:
my_dataframe = df_demo_lab_smoking_drugs_AntiH_Diab

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_Final_df.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

# Cardiovascular event and time

In [ ]:
import pandas
import os

# This query represents dataset "APOE_April2024" for domain "condition" and was generated for All of Us Controlled Tier Dataset v6
dataset_84056972_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (134057, 4023995, 4028244, 4028367)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 
                    UNION
                    DISTINCT SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_array_data = 1 ) )
                )
            ) c_occurrence 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                ON c_occurrence.condition_type_concept_id = c_type.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                ON v.visit_concept_id = visit.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_84056972_condition_df = pandas.read_gbq(
    dataset_84056972_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

df_APOE_person_CVD = pd.merge(df_APOE,dataset_84056972_condition_df,on="person_id",how="inner")
df_APOE_person_CVD['condition_start_datetime'] = df_APOE_person_CVD['condition_start_datetime'].dt.date

df_APOE_person_CVD.head(5)

In [ ]:
df_APOE_person_CVD = df_APOE_person_CVD[['person_id', 'standard_concept_name', 'condition_start_datetime']]

In [ ]:
df_APOE_person_CVD = df_APOE_person_CVD.groupby(['person_id']).agg({'condition_start_datetime': 'min'}).reset_index()
df_APOE_person_CVD = df_APOE_person_CVD.rename(columns={'condition_start_datetime': 'CVD_start_date'})
df_APOE_person_CVD['CVD'] = "1"
df_APOE_person_CVD = df_APOE_person_CVD.drop_duplicates()

In [ ]:
df_APOE_person_CVD.head()

In [ ]:
df_APOE_person_CVD = pd.merge(df_APOE,df_APOE_person_CVD,on="person_id",how="left")
df_APOE_person_CVD['CVD'] = df_APOE_person_CVD['CVD'].fillna("0")

In [ ]:
df_APOE_person_CVD.head()

In [ ]:
df = pd.read_csv('APOE_file_Final_df.csv').drop(columns=['APOE_group'])
df['Total_Cholesterol_time'] = pd.to_datetime(df['Total_Cholesterol_time'])
# Specify the common column for joining, assuming it's 'PersonID'
common_column = 'person_id'

# Perform inner joins
df_CVD = df.merge(df_APOE_person_CVD, on=common_column, how='left')

print(df_CVD.shape[0])
print(df_CVD['person_id'].nunique())

df_CVD.head()

In [ ]:
# Ensure the date columns are in datetime format
df_CVD['CVD_start_date'] = pd.to_datetime(df_CVD['CVD_start_date'])

# Apply the condition row-wise
df_CVD['CVD_at_timeLab'] = df_CVD.apply(
    lambda row: '1' if row['CVD_start_date'] >= row['Total_Cholesterol_time'] else '0',
    axis=1
)

print(df_CVD.shape[0])
print(df_CVD['person_id'].nunique())
df_CVD.head()

In [ ]:
my_dataframe = df_CVD

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'APOE_file_Final_df_CVD.csv'

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr